In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
def get_user_data(userid):
    url = "http://peace-data-team.ru:4001/user/games/steam"

    payload={}
    headers = {
      'steam_id': str(userid)
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return response.text



In [2]:
import json
def parse(response, data, user_id):
    data = data.append({'id':0}, ignore_index=True)
    data['id'][-1:] = str(user_id)
    r = json.loads(response)
    for i in range(r['response']['game_count']):
    
        game_data = r['response']['games'][i]
        if game_data['appid'] not in data.columns:
            data[game_data['appid']] = 0
        data[game_data['appid']][-1:] = game_data['playtime_forever']
    return data

In [3]:
def get_user_csgo_data(userid):
    url = "http://peace-data-team.ru:4001/user/stats/csgo"

    payload={}
    headers = {
      'steam_id': str(userid)
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return response.text

In [4]:
import time
counter = len(data)
exception_counter = 0
empty_counter = 0
for i in range(76561198000002456, 76561198100000000):
    try:
        response = get_user_data(i)
        exception_counter = 0
        if response == r'{"response":{}}':
            time.sleep(0.2)
            empty_counter += 1
            continue
        else:
            data = parse(response, data)
            counter += 1
            print(counter)
            if counter >= 1000:
                print('Done')
                break
    except Exception:
        exception_counter += 1
        i-=1
        if exception_counter > 10:
            print('kaput')
            break
        print('wait...')
        time.sleep(1)
        

NameError: name 'data' is not defined

In [15]:
game_data

,id,game,time
0,151603712,The Elder Scrolls V Skyrim,273.0
1,151603712,Fallout 4,87.0
2,151603712,Spore,14.9
3,151603712,Fallout New Vegas,12.1
4,151603712,Left 4 Dead 2,8.9
...,...,...,...
70484,128470551,Fallen Earth,2.4
70485,128470551,Magic Duels,2.2
70486,128470551,Titan Souls,1.5
70487,128470551,Grand Theft Auto Vice City,1.5


In [16]:
for i in range(len(game_data)):
    if i % 100 == 0:
        print(i)
    crosstab[game_data['game'].iloc[i]].loc[game_data['id'].iloc[i]] = game_data['time'][i]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [336]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

crosstab1 = crosstab.copy()
for col in crosstab.columns:
    if sum(crosstab[col]) < 100:
        crosstab1 = crosstab1.drop(col, axis=1)
        

train, test = train_test_split(crosstab1)



model = NearestNeighbors()
model.fit(train)

NearestNeighbors()

In [17]:
crosstab.to_csv('tab.csv')

In [ ]:
crosstab = pd.read_csv('tab.csv')

In [358]:
model.kneighbors([train.loc[151603712]])[1][0]

array([5608, 1572, 2945, 4338, 8030], dtype=int64)

In [18]:
import re
def getID64(id32):
    url = "https://steamid.xyz/" + str(id32)
    response = requests.request("GET", url).text
    res = re.sub('\n', '', response)
    res = re.search('Steam64 ID\r.*.*>\rNick Name', res).group(0)
    res = re.sub('Steam64', '', res)
    res = re.search('value=".*.*>', res).group(0)
    res = re.sub('value="', '', res)
    res = re.sub('">', '', res)
    return res

In [19]:
getID64(309903146)

'76561198270168874'

In [20]:
crosstab.index[0]

5250

In [21]:
data = pd.DataFrame({0: 0}, index=[0])

In [ ]:
exception_counter = 0
empty_counter = 0
counter = len(data)

for i in range(4967, len(crosstab.index)):
    try:
        user_id = getID64(crosstab.index[i])
        response = get_user_data(user_id)
        exception_counter = 0
        if response == r'{"response":{}}':
            time.sleep(0.2)
            empty_counter += 1
            continue
        else:
            data = parse(response, data, user_id)
            counter += 1
            print(counter)
            if counter >= 1500:
                print('Done')
                break
            if counter % 10 == 0:
                print('saved')
                data.to_csv('data.csv')
    except Exception:
        exception_counter += 1
        if exception_counter > 10:
            print('kaput')
            break
        print('wait...')
        time.sleep(1)

208
209
210
saved
211
212
213
214
215
216
217
218
219
wait...
220
saved
221
222
223
224
225
226
227
228
229
230
saved
231
232
233
234
235
236
237
238
239
240
saved
241
242
243
244
245
246
247
wait...
248
249
250
saved
251
252
wait...
253
254
255
256
257
258
259
260
saved
261
262
263
264
265
266
267
268
269
270
saved
271
272
273
274
275
276
277
278
279
280
saved
281
282
283
284
285
286
287
288
289
290
saved
291
292
293
294
295
296
297
298
299
300
saved
301
302
303
304
305
306
307
308
309
310
saved
311
312
313
314
315
316
317
318
319
320
saved
321
322
323
324
325
326
327
328
329
330
saved
wait...
331
332
333
334
335
336
337
338
339
340
saved
341
342
343
344
345
346
347
348
349
350
saved
351
352
353
354
355
356
357
358
359
360
saved
361
362
363
364
365
366
367
368
369
370
saved
371
372
373
374
375
376
377
378
379
380
saved
381
382
383
384
385
wait...
386
387


In [130]:
i

4967

In [17]:
data.to_csv('data_1.csv')

In [111]:
data = pd.read_csv('data.csv', index_col='id')
data

,Unnamed: 0,10,20,30,40,50,60,70,130,240,...,518030,449940,546700,555260,512200,245410,245430,4800,530930,397690
id,,,,,,,,,,,,,,,,,,,,,
76561197960979850,0,60811.0,0.0,0.0,0.0,0.0,0.0,60783.0,0.0,60865.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
76561197961630274,1,33173.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
76561197964942751,2,166.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
76561197968807932,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
76561197969331369,4,25.0,0.0,4.0,0.0,0.0,15.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198151226759,375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,392.0,246.0,242.0,0.0,0.0,0.0,0.0,0.0,0
76561198151469549,376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,13.0,0
76561198151551694,377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [112]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [30]:
data.drop(0, inplace=True)

In [113]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='constant', fill_value= 0)
c = data.columns
i = data.index
data = pd.DataFrame(imp.fit_transform(data))
data.columns = c
data.index = i

In [20]:
s = dict()
for c in data.columns[1:]:
    s[c] = sum(data[c])

In [114]:
data

,10,20,30,40,50,60,70,130,240,10540,...,518030,449940,546700,555260,512200,245410,245430,4800,530930,397690
id,,,,,,,,,,,,,,,,,,,,,
76561197960979850,60811.0,0.0,0.0,0.0,0.0,0.0,60783.0,0.0,60865.0,96.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197961630274,33173.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197964942751,166.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197968807932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197969331369,25.0,0.0,4.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198151226759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,392.0,246.0,242.0,0.0,0.0,0.0,0.0,0.0,0.0
76561198151469549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0
76561198151551694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
s = pd.Series(s)

In [196]:
new_data = data[s.sort_values().tail(1500).index]

In [197]:
new_data

,557340,871720,420110,249130,286360,205020,550320,41010,641990,328100,...,582660,238960,240,4000,252490,230410,252950,271590,578080,730
id,,,,,,,,,,,,,,,,,,,,,
76561197960979850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,60865.0,0.0,0.0,0.0,0.0,0.0,0.0,60784.0
76561197961630274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,156.0
76561197964942751,0.0,0.0,0.0,0.0,274.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,166.0
76561197968807932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561197969331369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198151226759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76561198151469549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,7.0,0.0,0.0,0.0,112.0,0.0,0.0,0.0,0.0
76561198151551694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [198]:
new_data['sum'] = new_data.sum(axis=1)

In [199]:
new_data = new_data[new_data['sum']>1200]

In [200]:
new_data.drop('sum', inplace=True, axis=1)

In [5]:
new_data = pd.read_csv('new_data.csv')

In [10]:
new_data = pd.read_csv('new_data.csv',index_col=0)

In [11]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=20)
model.fit(new_data)

NearestNeighbors(n_neighbors=20)

In [12]:
import json
def parse(response, data, user_id):
    data = data.append({'id':0}, ignore_index=True)
    data['id'][-1:] = str(user_id)
    r = json.loads(response)
    for i in range(r['response']['game_count']):
    
        game_data = r['response']['games'][i]
        if game_data['appid'] not in data.columns:
            data[game_data['appid']] = 0
        data[game_data['appid']][-1:] = game_data['playtime_forever']
    return data

def new_parse(user_id):
    res = get_user_data(user_id)
    data = pd.DataFrame({'id':0}, index=[0])
    r = json.loads(res)
    for i in range(r['response']['game_count']):
        game_data = r['response']['games'][i]
        if game_data['appid'] not in data.columns:
            data[game_data['appid']] = 0
        data[game_data['appid']][-1:] = game_data['playtime_forever']
    data['id'][-1:] = user_id
    return data.set_index('id', drop=True)

In [13]:
def get_recomendation_games(user_id):
    user_res = new_parse(user_id)
    user_col = user_res.columns
    c = new_data.columns
    cols = list(set(user_col) & set(c))
    zeros =  list(set(c) - set(user_col))
    user_res[zeros] = 0
    user_res = user_res[c]
    res = model.kneighbors(user_res, 5)[1][0]
    recomendations = new_data.iloc[res][zeros].sum(axis=0).sort_values().tail(10)
    
    return pd.DataFrame((pd.DataFrame(recomendations).index)).to_json()

In [ ]:
get_recomendation_games(76561197972876528)

In [206]:
new_data.to_csv('new_data.csv')

In [207]:
import pickle

pickle.dump(model, open('model.sav', 'wb'))

In [134]:
def get_user_csgo_data(userid):
    url = "http://peace-data-team.ru:4001/user/stats/csgo"

    payload={}
    headers = {
      'steam_id': str(userid)
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    return response.text

In [223]:
def get_recomendation_users(user_id):
    global new_data, model
    user_res = new_parse(user_id)
    user_col = user_res.columns
    c = new_data.columns
    cols = list(set(user_col) & set(c))
    zeros = list(set(c) - set(user_col))
    user_res[zeros] = 0
    user_res = user_res[c]
    res = model.kneighbors(user_res)[1][0]
    recomendations = new_data.iloc[res]
    return user_col
    return pd.DataFrame(new_data.iloc[res].index[1:])

In [226]:
set(get_recomendation_users(76561197972876528)) & set(new_data.columns)

set()

In [9]:
get_recomendation_users(76561197972876528)

NameError: name 'get_recomendation_users' is not defined

In [209]:
get_recomendation_games(76561197972876528)

'{"0":{"0":"477160","1":"312990","2":"1046930","3":"386360","4":"476480","5":"217750","6":"1097150","7":"359550","8":"32800","9":"578080"}}'

In [208]:
new_data

,557340,871720,420110,249130,286360,205020,550320,41010,641990,328100,...,582660,238960,240,4000,252490,230410,252950,271590,578080,730
id,,,,,,,,,,,,,,,,,,,,,
76561197960979850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,60865.0,0.0,0.0,0.0,0.0,0.0,0.0,60784.0
76561197961630274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,156.0
76561197964942751,0.0,0.0,0.0,0.0,274.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,166.0
76561197972206066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,21.0,13.0,0.0,0.0,1002.0,2804.0,7516.0,3768.0
76561197972876528,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,107.0,47683.0,0.0,0.0,0.0,0.0,160.0,168.0,1016.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76561198150195309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,22.0,0.0,30314.0,82641.0,52158.0
76561198151019827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,94991.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1708.0
76561198151226759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
